<a href="https://colab.research.google.com/github/AlinaZakharova1997/Tokenizator/blob/master/Topic_model_gensim_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Тематическое моделирование

Задача тематического моделирования состоит в том, чтобы разложить слова, употребляющиеся в корпусе на темы и приписать эти темы каждому документу в корпусе.

Помимо анализа тематического наполнения корпуса, тематическое моделирование может использоваться для:

1) построения вектроных представлений текстов (числа показывают насколько тема выражена в тексте). Такие представления могут быть лучше обычных (TfidfVectorizer, CountVectorizer), т.к. позволяют находить близкие документы даже если в них нет одинаковых слов (но есть слова одной тематики). Тематические векторы могут использоваться для поиска, рекомендаций или как признаки в классификации. 

2) приписывания тэгов текстам. Так можно автоматически выделять в текстовых потоках тренды, горячие темы. Тэги можно использовать как фильтр в поисковых системах. Тэги нужно приписывать темам вручную, но это проще чем размечать обучающую выборку, так как слова в тематике сразу подказывают название. 

3) составления тематических словарей



Кроме обычных текстов тематическое моделирование применяется к многим другим данным: запросам пользователей, сайтам, на которые заходят пользователи, музыке, которую слушают пользователи, покупкам в одной корзине, банковским транзакциям и даже днк. 


Все подходы к тематическому моделированию так или иначе основнованы на:

1) модели мешка слов (т.е. порядок слов в документах не учитывается)  
2) независимости документов между собой  (т.е. употребление слова W в тексте D_1 никак не влияет на слова в документе D_2)  
3) дистрибутивной гипотезе (слова, употребляющиеся вместе, объединяются в темы)



В этой тетрадке для получения тематических моделей используются LDA из gensim и NMF из sklearn.

Про LDA (и в целом тематическое моделирование) можно почитать вот эту статью - https://sysblok.ru/knowhow/kak-ponjat-o-chem-tekst-ne-chitaja-ego/

In [ ]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#! pip3 install pymorphy2
from pymorphy2 import MorphAnalyzer
#! pip3 install pyLDAvis
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
from IPython.core.display import HTML 
morph = MorphAnalyzer()
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Данные

Возьмем 10 тыс статьи с Википедии. 

Токенизируем самым простым способом и нормализуем Pymorphy.

In [ ]:

stops = set(stopwords.words('russian'))
also_stops = ['свой', 'это','также','мочь','однако', 'иметь', 'быть', 'стать']
# чтобы быстрее нормализовать тексты, создадим словарь всех словоформ
# нормазуем каждую 1 раз и положим в словарь
# затем пройдем по текстам и сопоставим каждой словоформе её нормальную форму

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    clean = []
    for word, _ in uniq.most_common(top):
        if not re.search('[\'\"#\—\»i\«]+?',word) and not re.search('[a-zA-Z]+?',word) and not re.search('[0-9]+?',word):
            clean.append((word,_))
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in clean}
    norm_texts = []
    for text in texts:
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops and word not in also_stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [67]:
corpus = open('wiki_data.txt').read().splitlines()[:10000]

In [68]:
corpus = opt_normalize([tokenize(text.lower()) for text in corpus], 30000)

In [36]:
corpus[:3]

[['нижегородский',
  'сельский',
  'посёлок',
  'район',
  'нижегородский',
  'область',
  'входить',
  'состав',
  'расположить',
  'км',
  'юг',
  'село',
  'км',
  'запад',
  'город',
  'право',
  'берег',
  'река',
  'правый',
  'приток',
  'река',
  'сатис',
  'окружить',
  'смешанный',
  'лес',
  'соединить',
  'дорога',
  'посёлок',
  'км',
  'дорога',
  'посёлок',
  'сатис',
  'км',
  'название',
  'являться',
  'сугубо',
  'официальный',
  'местный',
  'население',
  'использовать',
  'исключительно',
  'название',
  'употребляться',
  'языковой',
  'оборот',
  'ранее',
  'использовать',
  'название',
  'год',
  'переселенец',
  'соседний',
  'село',
  'аламасовый',
  'расположить',
  'соответственно',
  'км',
  'запад',
  'посёлок',
  'жить',
  'рабочий',
  'совхоз',
  'центр',
  'посёлок',
  'сатис',
  'возле',
  'посёлок',
  'расположить',
  'активно',
  'камень',
  'настоящий',
  'время',
  'официально',
  'данные',
  'год',
  'посёлок',
  'насчитываться',
  'хозяйство',
 

In [69]:

ph = gensim.models.Phrases(corpus, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[corpus]



ph = gensim.models.Phrases(corpus, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[corpus]

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [77]:
#dictinary = gensim.corpora.Dictionary(texts)
mydictinary = gensim.corpora.Dictionary(ngrammed_texts)

In [78]:
mydictinary.filter_extremes(no_above=0.4, no_below=40)
mydictinary.compactify()

In [79]:
print(mydictinary)

Dictionary(3156 unique tokens: ['активно', 'благодаря', 'возле', 'возраст', 'входить_состав']...)


Преобразуем наши тексты в мешки слов. 

In [83]:
corpus = [mydictinary.doc2bow(text) for text in ngrammed_texts]
tfidf = gensim.models.TfidfModel(corpus, id2word=mydictinary)
corpus = tfidf[corpus]

Про параметры можно почитать в документации:

In [84]:
?gensim.models.LdaMulticore

Основные это num_topics, alpha, eta и passes. 

**num_topics** - это количество тем. Это основной параметр и настраивать его проще всего. Обычно 200 оптимальное значение. Можно поставить поменьше, если тексты не очень разнообразные или хочется уменьшить время обучения.

**alpha** и **eta** - параметры, которые влияют на разреженность распределения документы-темы и темы-слова. У alpha есть значения "asymmetric" и "auto", которые можно попробовать (по умолчанию стоит "symmetric", т.е. не разреженное). Eta можно задать каким-то числом или самому сделать изначальное распределение слов по темам. НО настраивать эти параметры сложно и непонятно и вообще лучше надеяться, что по умолчанию все заработает.

**passes** - задает количество проходов по данным. Чем больше, тем лучше сойдется модель, но обучаться будет дольше.

In [85]:
lda = gensim.models.LdaMulticore(corpus, 15, id2word=dictinary, passes=15) # если поддерживается многопоточность
# lda = gensim.models.LdaModel(200, id2word=dictinary, passes=5)

Посмотрим на топики.

In [86]:
lda.print_topics()


[(0,
  '0.063*"станция" + 0.019*"географический" + 0.017*"носитель" + 0.016*"украина" + 0.012*"линия" + 0.012*"дания" + 0.012*"юрий" + 0.011*"лев" + 0.011*"парк" + 0.010*"венгрия"'),
 (1,
  '0.048*"литература" + 0.019*"гитарист" + 0.009*"производственный" + 0.009*"выпускаться" + 0.007*"технический" + 0.005*"соединить_штат" + 0.005*"список" + 0.003*"включить" + 0.003*"прага" + 0.003*"страница"'),
 (2,
  '0.016*"фильм" + 0.013*"альбом" + 0.011*"песня" + 0.008*"группа" + 0.007*"фамилия" + 0.006*"роль" + 0.006*"который" + 0.006*"музыка" + 0.006*"американский" + 0.005*"выпустить"'),
 (3,
  '0.005*"г" + 0.004*"университет" + 0.004*"который" + 0.003*"сын" + 0.003*"церковь" + 0.003*"имя" + 0.003*"город" + 0.003*"школа" + 0.003*"член" + 0.003*"время"'),
 (4,
  '0.024*"команда" + 0.023*"клуб" + 0.021*"сезон" + 0.019*"турнир" + 0.015*"матч" + 0.015*"чемпионат" + 0.012*"чемпионат_мир" + 0.011*"сборная" + 0.010*"выступать" + 0.010*"находиться_емильчинский"'),
 (5,
  '0.043*"летний_олимпийский" + 0.

ТОП-3 нормальных тем по моей версии без тдф:
1. (14,
  '0.022*"армия" + 0.018*"войско" + 0.016*"самолёт" + 0.016*"немецкий" + 0.015*"город" + 0.015*"часть" + 0.013*"бой" + 0.013*"противник" + 0.013*"район" + 0.009*"фронт"')]
2.  (2,
  '0.076*"перепись_год" + 0.071*"житомирский_область" + 0.071*"составлять_человек" + 0.070*"занимать_площадь" + 0.070*"почтовый_индекс" + 0.070*"телефонный_код" + 0.069*"район_житомирский" + 0.055*"коатуа_население" + 0.054*"тело" + 0.054*"село_украина"')
3. (13,
  '0.014*"партия" + 0.013*"член" + 0.011*"г" + 0.010*"президент" + 0.010*"совет" + 0.010*"правительство" + 0.010*"право" + 0.009*"страна" + 0.009*"национальный" + 0.009*"организация"')


  ТОП-3 нормальных тем с тдф:
  1.(8,
  '0.052*"село" + 0.021*"посёлок" + 0.018*"район" + 0.018*"хутор" + 0.016*"сельский" + 0.015*"сельский_поселение" + 0.015*"поселение" + 0.015*"река" + 0.014*"район_ростовский" + 0.013*"ростовский_область"')
  2. (12,
  '0.063*"житомирский_область" + 0.063*"почтовый_индекс" + 0.063*"телефонный_код" + 0.063*"район_житомирский" + 0.063*"занимать_площадь" + 0.062*"составлять_человек" + 0.060*"перепись_год" + 0.057*"село_украина" + 0.050*"коатуа_население" + 0.045*"тело"')
  3. (5,
  '0.043*"летний_олимпийский" + 0.036*"игра" + 0.026*"олимпийский_игра" + 0.024*"следующий_раунд" + 0.021*"спортсмен" + 0.019*"проходить" + 0.015*"каждый_заезд" + 0.014*"финал" + 0.014*"соревнование" + 0.014*"принимать_участие"')
  

Ещё есть штука для визуализации.

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda, corpus, dictinary)

На графике должно быть как можно меньше пересекающихся кружков (т.е. темы состоят из разных слов), а сами кружки не должны быть огромными (скорее всего такую тему можно разбить на несколько поменьше).

Можно посмотреть метрики.

In [45]:
import numpy as np

Перплексия показывает насколько хороше моделируется корпус. Чем ближе к нулю, тем лучше. Можно использовать, чтобы настраивать количество проходов по корпусу (когда перестало улучшаться, то можно останавливаться).

In [87]:
lda.log_perplexity(corpus[:1000])
# без тдф -7.663149147828475

-12.352014566209919

Ещё есть когерентность. Она численно оценивает качество тем (проверяется, что темы состоят из разных слов и что в теме есть топ тематических слов). 

In [88]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                  texts=corpus, 
                                                   dictionary=dictinary, coherence='c_v')
#print(corpus[:10])

Чем выше, тем лучше.

In [89]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [90]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=corpus, 
                                                   dictionary=dictinary, coherence='c_v')

In [91]:
coherence_model_lda.get_coherence()
# без тдф nan

nan

Но все эти числа вспомогательны! Главные критерии качества модели: интерпретируемость и понятность тем (т.е. нужно глазами смотреть на каждую тему), а также польза для практической задачи, которую вы пытаетесь решить.

### Разложение матриц в sklearn

In [ ]:
Image(url="https://www.researchgate.net/profile/Andrea_Bertozzi/publication/312157184/figure/fig1/AS:448453387001860@1483931027472/Conceptual-illustration-of-non-negative-matrix-factorization-NMF-decomposition-of-a.png",
     width=800, height=500)


NMF - превращает одну матрицу Words * Documents в произведение двух матриц Words * Topics и Topics * Documents (произведение не точно равно изначальной матрице, но достаточно близко - чем больше Topics, тем точнее, но больше тратиться памяти и времени). 

Таким образом, взяв одну из получившихся матриц, мы получим или тематические представления документов (вторая матрица - документы на темы), либо слова, разложенные по темам (первая матрица - темы на слова).

Отличие от LDA в том, что числа в матрицах не будут вероятностями, а просто каким-то положительными числам. Чем больше, тем сильнее слово или документ связан с темой. 

(Почему и как это все работает, вы можете почитать отдельно. Для практических задач хватит умения запускать все в sklearn) 

In [54]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [57]:
corpus_text = open('wiki_data.txt').read().splitlines()[:10000]
texts = opt_normalize([tokenize(text.lower()) for text in corpus_text], 30000)
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [58]:
vectorizer = TfidfVectorizer(max_features=1500, min_df=40, max_df=0.4, ngram_range=(1,2))
X = vectorizer.fit_transform(stexts)

Разложим её.

In [59]:
# n_components - главный параметр в NMF, это количество тем. 
# Если данных много, то увеличения этого параметра сильно увеличивает время обучения
model = NMF(n_components=15)

In [60]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=15, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [61]:
model.components_.shape # матрица темы на слова

(15, 1500)

In [62]:
model.transform(X).shape # матрица документы на темы

(10000, 15)

In [63]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

Вытащим словарь, по которому мы построили модель.

In [64]:
feat_names = vectorizer.get_feature_names()

Теперь посмотрим на матрицу темы-слова, отсортируем её по строкам и возьмем топ N слов, сопоставив индексы со словарём

In [65]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "--".join(words))

0 житомирский--житомирский область--район житомирский--почтовый индекс
1 летний олимпийский--летний--олимпийский--олимпийский игра
2 который--время--город--её
3 село--км--расстояние км--река
4 вид--семейство--подсемейство--длина
5 зимний олимпийский--зимний--олимпийский--олимпийский игра
6 клуб--чемпионат--матч--команда
7 хутор--ростовский--район ростовский--сельский
8 альбом--песня--группа--выпустить
9 остров--озеро--расположить--метр
10 война--армия--советский--военный
11 новоград--новоград волынский--волынский--житомирский
12 уезд--волость--состав--округ
13 населить пункт--населить--пункт--название
14 фильм--роль--режиссёр--актёр


У разложения есть метрика, показывающая насколько хорошо восстанавливается изначальная матрица. Чем меньше, тем лучше.

In [66]:
model.reconstruction_err_

83.6748955262086

ТОП-3 тем по моей версии:

1.клуб--чемпионат--матч--команда
2.альбом--песня--группа--выпустить
3.фильм--роль--режиссёр--актёр

Но как и с LDA - главное это польза от модели и человеческая оценка, получаемых тем.

## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); - done

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы);- done 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); - done

4) постройте несколько LDA моделей (переберите количество тем, можете поменять alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; - done

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы; - done

6) между словарем и обучением модели добавьте tfidf (`tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`); done

7) повторите пункт 4 на преобразованном корпусе (подбирайте параметры, ориентируясь на качество, а не на результаты, которые вы получали без tfidf);-done

8) в отдельной ячейке сравните лучшую модель без tfidf и лучшую модель с tfidf (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)- done

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем - done

10) в отдельной ячейки напечатайте темы лучшей NMF модели, сравните их с теми, что получились в LDA.- done

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **


Острожнее интерпретируйте полученные результаты. Если один алгоритм сработал хорошо в этом задании - не значит, что он всегда будет хорошо работать, и наоборот.